In [ ]:
# Install the required packages
%pip install -qU anthropic klavis requests


In [ ]:
import os
import json
from anthropic import Anthropic
from klavis import Klavis
from klavis.types import McpServerName, ConnectionType, ToolFormat

# Set environment variables
os.environ["ANTHROPIC_API_KEY"] = "YOUR_ANTHROPIC_API_KEY"  # Replace with your actual Anthropic API key
os.environ["KLAVIS_API_KEY"] = "YOUR_KLAVIS_API_KEY"  # Replace with your actual Klavis API key


In [ ]:
klavis_client = Klavis(api_key=os.getenv("KLAVIS_API_KEY"))

youtube_mcp_instance = klavis_client.mcp_server.create_server_instance(
    server_name=McpServerName.YOUTUBE,
    user_id="1234",
    platform_name="Klavis",
    connection_type=ConnectionType.STREAMABLE_HTTP,
)

# print(f"🔗 YouTube MCP server created at: {youtube_mcp_instance.server_url}, and the instance id is {youtube_mcp_instance.instance_id}")


In [ ]:
def claude_with_mcp_server(mcp_server_url: str, user_query: str):
    claude_client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

    messages = [
        {"role": "user", "content": f"{user_query}"}
    ]
    
    mcp_server_tools = klavis_client.mcp_server.list_tools(
        server_url=mcp_server_url,
        connection_type=ConnectionType.STREAMABLE_HTTP,
        format=ToolFormat.ANTHROPIC,
    )
    
    response = claude_client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=4000,
        system="You are a helpful assistant. Use the available tools to answer the user's question.",
        messages=messages,
        tools=mcp_server_tools.tools
    )
    
    messages.append({"role": "assistant", "content": response.content})

    if response.stop_reason == "tool_use":
        tool_results = []
        
        for content_block in response.content:
            if content_block.type == "tool_use":
                function_name = content_block.name
                function_args = content_block.input
                
                print(f"🔧 Calling: {function_name}, with args: {function_args}")
                
                result = klavis_client.mcp_server.call_tools(
                    server_url=mcp_server_url,
                    tool_name=function_name,
                    tool_args=function_args,
                    connection_type=ConnectionType.STREAMABLE_HTTP
                )
                
                tool_results.append({
                    "type": "tool_result",
                    "tool_use_id": content_block.id,
                    "content": result.result.content[0]['text']
                })
        
        messages.append({"role": "user", "content": tool_results})
            
        final_response = claude_client.messages.create(
            model="claude-3-5-sonnet-20241022",
            max_tokens=4000,
            system="You are a helpful assistant. Use the available tools to answer the user's question.",
            messages=messages,
            tools=mcp_server_tools.tools
        )
        
        return final_response.content[0].text
    else:
        return response.content[0].text


In [ ]:
YOUTUBE_VIDEO_URL = "https://www.youtube.com/watch?v=LCEmiRjPEtQ"  # pick a video you like!

result = claude_with_mcp_server(
    mcp_server_url=youtube_mcp_instance.server_url, 
    user_query=f"Please provide a complete summary of this YouTube video with timestamp: {YOUTUBE_VIDEO_URL}"
)

print(result)


In [ ]:
import webbrowser

gmail_mcp_server = klavis_client.mcp_server.create_server_instance(
    server_name=McpServerName.GMAIL,
    user_id="1234",
    platform_name="Klavis",
    connection_type=ConnectionType.STREAMABLE_HTTP,
)

webbrowser.open(gmail_mcp_server.oauth_url)

print(f"🔐 Opening OAuth authorization for Gmail, if you are not redirected, please open the following URL in your browser: {gmail_mcp_server.oauth_url}")


In [ ]:
EMAIL_RECIPIENT = "zihaolin@klavis.ai" # Replace with your email
EMAIL_SUBJECT = "Test Claude + Gmail MCP Server"
EMAIL_BODY = "Hello World from Claude!"

result = claude_with_mcp_server(
    mcp_server_url=gmail_mcp_server.server_url, 
    user_query=f"Please send an email to {EMAIL_RECIPIENT} with subject {EMAIL_SUBJECT} and body {EMAIL_BODY}"
)

print(result)
